# **Ensemble learning** - master thesis


Four models outputs voting on *is change or not?*

In [ ]:
import numpy as np
import cv2
from collections import Counter

def kappa(tp, tn, fp, fn):
    N = tp + tn + fp + fn
    p0 = (tp + tn) / N
    pe = ((tp+fp)*(tp+fn) + (tn+fp)*(tn+fn)) / (N * N)

    return (p0 - pe) / (1 - pe)

# Define color constants
BLACK = (0, 0, 0)  # True Negative
WHITE = (255, 255, 255)  # True Positive
MAGENTA = (255, 0, 255)  # False Negative
GREEN = (0, 255, 0)  # False Positive

# Function to convert color to class
def color_to_class(color):
    if np.array_equal(color, BLACK):
        return 0  # True Negative
    elif np.array_equal(color, WHITE):
        return 1  # True Positive
    elif np.array_equal(color, MAGENTA):
        return 0  # False Negative | orginally 2
    elif np.array_equal(color, GREEN):
        return 1  # False Positive \ originally 3
    else:
        raise ValueError("Unknown color")

# Function to convert class to color
def class_to_color(cls):
    if cls == 0:
        return BLACK
    elif cls == 1:
        return WHITE
    elif cls == 2:
        return MAGENTA
    elif cls == 3:
        return GREEN
    else:
        raise ValueError("Unknown class")

# Read images (change maps) from the models
image_paths = ["FC-EF-dubai.png", "FC-Siam-conc-dubai.png", "FC-Siam-diff-dubai.png", "UCDNet-dubai.png"]
#image_paths = ["FC-EF-lasvegas.png", "FC-Siam-conc-lasvegas.png", "FC-Siam-diff-lasvegas.png", "UCDNet-lasvegas.png"]
#image_paths = ["FC-EF-montpellier.png", "FC-Siam-conc-montpellier.png", "FC-Siam-diff-montpellier.png", "UCDNet-montpellier.png"]
#image_paths = ["FC-EF-rio.png", "FC-Siam-conc-rio.png", "FC-Siam-diff-rio.png", "UCDNet-rio.png"]
images = [cv2.imread(path) for path in image_paths]

# Read the ground truth change map
ground_truth_path = "cm.png"
ground_truth = cv2.imread(ground_truth_path)

#FC-EF, FC-Siam-conc, FC-Siam-diff and UCDNet
# the UCDNet original network loss is -0.91
# Assume network losses for the models (lower is better)
network_losses = [0.44, 0.36, 0.41, -0.91]
normalized_losses = [0.296,0.286,0.292,0.126]

# Normalize the losses to use as weights
weights = [1.0 / loss if loss > 0 else 1.0 for loss in normalized_losses]
weights = np.array(weights) / sum(weights)

# Get the dimensions of the image
height, width, _ = images[0].shape

# Initialize the final change map
final_change_map = np.zeros_like(images[0])

# Initialize lists to store true labels and predicted labels
true_labels = []
pred_labels = []

tp = 0
tn = 0
fp = 0
fn = 0


# Ensemble the results
for y in range(height):
    for x in range(width):
        # Get the predictions from all models for the current pixel
        pixel_predictions = [color_to_class(images[i][y, x]) for i in range(4)]

        # Weighted voting
        vote_counts = Counter()
        for i, pred in enumerate(pixel_predictions):
            vote_counts[pred] += weights[i]

        # Get the class with the highest vote
        final_class = max(vote_counts, key=vote_counts.get)

        # Store the true and predicted labels
        true_labels.append(color_to_class(ground_truth[y, x]))
        pred_labels.append(final_class)

        true_tmp = color_to_class(ground_truth[y, x])
        predicted_tmp = final_class

        final_class_prediction = 0
        if true_tmp == 0 and predicted_tmp == 0:
          final_class_prediction = 0
          tn += 1
        elif true_tmp == 1 and predicted_tmp == 1:
          final_class_prediction = 1
          tp += 1
        elif true_tmp == 1 and predicted_tmp == 0:
          final_class_prediction = 2
          fn += 1
        elif true_tmp == 0 and predicted_tmp == 1:
          final_class_prediction = 3
          fp += 1

        # Set the final pixel value
        final_change_map[y, x] = class_to_color(final_class_prediction)

# Save the final change map
cv2.imwrite("final_change_map.png", final_change_map)

print("Final change map created and saved as 'final_change_map.png'")

#My metrics
accuracy = (tp + tn)/(tp + tn + fn + fp)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
dice = 2 * precision * recall / (precision + recall)
k = kappa(tp, tn, fp, fn)

print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score (Dice): {dice:.2f}")
print(f"Kappa: {k:.2f}")


Final change map created and saved as 'final_change_map.png'
Accuracy: 0.92
Precision: 0.75
Recall: 0.28
F1 Score (Dice): 0.44
Kappa: 0.37
